# Dataset Time Horizon Checker

This notebook analyzes all datasets across the 6 category folders to identify their time coverage (min/max years). This helps identify which datasets have limited time horizons and may need updating.


In [ ]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import glob
import warnings
warnings.filterwarnings('ignore')


## Helper Functions


In [ ]:
def check_dataset_time_range(filepath):
    """
    Check the time range (min/max years) of a dataset file.
    
    Args:
        filepath: Path to the CSV file
    
    Returns:
        dict with min_year, max_year, data_points, and status
    """
    try:
        df = pd.read_csv(filepath)
        
        if 'time' not in df.columns:
            return {
                'min_year': None,
                'max_year': None,
                'data_points': 0,
                'status': 'Error: No time column'
            }
        
        if len(df) == 0:
            return {
                'min_year': None,
                'max_year': None,
                'data_points': 0,
                'status': 'Error: Empty file'
            }
        
        min_year = int(df['time'].min())
        max_year = int(df['time'].max())
        data_points = len(df)
        
        # Determine status based on max year
        current_year = 2025
        years_behind = current_year - max_year
        
        if max_year >= 2024:
            status = 'Up to date'
        elif max_year >= 2020:
            status = f'{years_behind} years behind'
        elif max_year >= 2015:
            status = f'{years_behind} years behind (Limited)'
        else:
            status = f'{years_behind} years behind (Very Limited)'
        
        return {
            'min_year': min_year,
            'max_year': max_year,
            'data_points': data_points,
            'status': status
        }
    
    except Exception as e:
        return {
            'min_year': None,
            'max_year': None,
            'data_points': 0,
            'status': f'Error: {str(e)}'
        }


def extract_indicator_name(filename):
    """
    Extract indicator name from filename.
    Pattern: ddf--datapoints--{indicator}--by--geo--time.csv
    """
    if 'ddf--datapoints--' in filename and '--by--geo--time.csv' in filename:
        indicator = filename.replace('ddf--datapoints--', '').replace('--by--geo--time.csv', '')
        return indicator
    return filename.replace('.csv', '')


## Main Analysis Function


In [ ]:
def analyze_all_datasets(base_dir='./data/gapminder'):
    """
    Analyze all datasets in category folders and return a summary DataFrame.
    
    Args:
        base_dir: Base directory containing category folders
    
    Returns:
        DataFrame with time horizon information for all datasets
    """
    base_path = Path(base_dir)
    categories = ["Physical", "Mental", "Social", "Economic", "Environmental", "Cultural"]
    
    results = []
    
    for category in categories:
        category_path = base_path / category
        
        if not category_path.exists():
            print(f"Warning: Category folder '{category}' not found")
            continue
        
        # Find all CSV files in the category folder
        csv_files = list(category_path.glob('ddf--datapoints--*.csv'))
        
        if len(csv_files) == 0:
            print(f"Warning: No datasets found in '{category}' folder")
            continue
        
        print(f"Analyzing {len(csv_files)} datasets in '{category}'...")
        
        for filepath in csv_files:
            indicator = extract_indicator_name(filepath.name)
            time_info = check_dataset_time_range(filepath)
            
            results.append({
                'Category': category,
                'Dataset': indicator,
                'Min Year': time_info['min_year'],
                'Max Year': time_info['max_year'],
                'Year Range': f"{time_info['min_year']}-{time_info['max_year']}" if time_info['min_year'] and time_info['max_year'] else 'N/A',
                'Data Points': time_info['data_points'],
                'Status': time_info['status']
            })
    
    df = pd.DataFrame(results)
    return df


# Run the analysis
print("=" * 70)
print("Analyzing Time Horizons for All Datasets")
print("=" * 70)
print()

results_df = analyze_all_datasets()

print(f"\n✅ Analysis complete! Found {len(results_df)} datasets across {results_df['Category'].nunique()} categories.")


In [ ]:
# Display results grouped by category
categories = ["Physical", "Mental", "Social", "Economic", "Environmental", "Cultural"]

for category in categories:
    category_data = results_df[results_df['Category'] == category]
    
    if len(category_data) == 0:
        continue
    
    print(f"\n{'='*70}")
    print(f"{category.upper()} ({len(category_data)} datasets)")
    print(f"{'='*70}")
    
    # Display table
    display_cols = ['Dataset', 'Min Year', 'Max Year', 'Year Range', 'Data Points', 'Status']
    print(category_data[display_cols].to_string(index=False))
    
    # Summary for this category
    if category_data['Max Year'].notna().any():
        avg_max_year = category_data['Max Year'].mean()
        min_max_year = category_data['Max Year'].min()
        max_max_year = category_data['Max Year'].max()
        print(f"\n  Summary: Average max year = {avg_max_year:.0f}, Range = {min_max_year:.0f}-{max_max_year:.0f}")


## Overall Statistics


In [ ]:
# Overall statistics
print(f"\n{'='*70}")
print("OVERALL STATISTICS")
print(f"{'='*70}\n")

# Count datasets by max year
print("Datasets by Maximum Year:")
max_year_counts = results_df['Max Year'].value_counts().sort_index(ascending=False)
for year, count in max_year_counts.head(10).items():
    if pd.notna(year):
        print(f"  {int(year)}: {count} dataset(s)")

# Count by status
print(f"\nDatasets by Status:")
status_counts = results_df['Status'].value_counts()
for status, count in status_counts.items():
    print(f"  {status}: {count} dataset(s)")

# Summary statistics
print(f"\nSummary Statistics:")
print(f"  Total datasets analyzed: {len(results_df)}")
print(f"  Total categories: {results_df['Category'].nunique()}")

if results_df['Max Year'].notna().any():
    print(f"  Earliest max year: {int(results_df['Max Year'].min())}")
    print(f"  Latest max year: {int(results_df['Max Year'].max())}")
    print(f"  Average max year: {results_df['Max Year'].mean():.1f}")
    print(f"  Median max year: {results_df['Max Year'].median():.0f}")

# Count datasets that need updating (max year < 2024)
needs_update = results_df[results_df['Max Year'] < 2024]
print(f"\n  Datasets needing updates (max year < 2024): {len(needs_update)}")

# Count datasets up to date (max year >= 2024)
up_to_date = results_df[results_df['Max Year'] >= 2024]
print(f"  Datasets up to date (max year >= 2024): {len(up_to_date)}")

print(f"\n{'='*70}")


## Datasets Needing Updates

Datasets with max year < 2024 that may need updating:


In [ ]:
# Show datasets that need updating
needs_update = results_df[results_df['Max Year'] < 2024].copy()

if len(needs_update) > 0:
    # Sort by max year (oldest first)
    needs_update = needs_update.sort_values('Max Year')
    
    print(f"\nFound {len(needs_update)} datasets with max year < 2024:\n")
    display_cols = ['Category', 'Dataset', 'Max Year', 'Status']
    print(needs_update[display_cols].to_string(index=False))
    
    # Group by max year
    print(f"\n\nGrouped by Max Year:")
    for max_year in sorted(needs_update['Max Year'].unique(), reverse=True):
        datasets = needs_update[needs_update['Max Year'] == max_year]
        print(f"\n  Max Year = {int(max_year)} ({len(datasets)} dataset(s)):")
        for _, row in datasets.iterrows():
            print(f"    - {row['Category']}/{row['Dataset']}")
else:
    print("\n✅ All datasets are up to date (max year >= 2024)!")


## Visualization: Maximum Year by Dataset


In [ ]:
import matplotlib.pyplot as plt

# Create visualization
fig, axes = plt.subplots(2, 1, figsize=(14, 12))

# Plot 1: Bar chart of max year by dataset
ax1 = axes[0]
results_sorted = results_df.sort_values('Max Year', na_position='last')
colors = ['green' if y >= 2024 else 'orange' if y >= 2020 else 'red' if pd.notna(y) else 'gray' 
          for y in results_sorted['Max Year']]

bars = ax1.barh(range(len(results_sorted)), results_sorted['Max Year'], color=colors, alpha=0.7)
ax1.set_yticks(range(len(results_sorted)))
ax1.set_yticklabels([f"{row['Category']}/{row['Dataset']}" for _, row in results_sorted.iterrows()], fontsize=9)
ax1.set_xlabel('Maximum Year', fontsize=12, fontweight='bold')
ax1.set_title('Maximum Year by Dataset\n(Green: ≥2024, Orange: 2020-2023, Red: <2020)', 
              fontsize=13, fontweight='bold', pad=15)
ax1.axvline(x=2024, color='blue', linestyle='--', alpha=0.5, label='Target (2024)')
ax1.axvline(x=2025, color='purple', linestyle='--', alpha=0.5, label='Goal (2025)')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='x')

# Plot 2: Distribution of max years
ax2 = axes[1]
valid_years = results_df['Max Year'].dropna()
if len(valid_years) > 0:
    ax2.hist(valid_years, bins=range(int(valid_years.min()), int(valid_years.max())+2), 
             edgecolor='black', alpha=0.7, color='steelblue')
    ax2.axvline(x=2024, color='red', linestyle='--', linewidth=2, label='Target (2024)')
    ax2.axvline(x=2025, color='purple', linestyle='--', linewidth=2, label='Goal (2025)')
    ax2.set_xlabel('Maximum Year', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Number of Datasets', fontsize=12, fontweight='bold')
    ax2.set_title('Distribution of Maximum Years Across All Datasets', 
                  fontsize=13, fontweight='bold', pad=15)
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print summary
print(f"\nColor Legend:")
print(f"  Green: Up to date (max year ≥ 2024)")
print(f"  Orange: Recent but needs update (max year 2020-2023)")
print(f"  Red: Limited coverage (max year < 2020)")
print(f"  Gray: Error or missing data")
